In [26]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import kagglehub

In [27]:
def plotting(S, energy_threshold,img, image_name,folder):
    total_energy = np.sum(S**2)
    cumulative_energy = np.cumsum(S**2) / total_energy
    
    # Determina il numero di valori singolari necessari
    k = np.argmax(cumulative_energy >= energy_threshold) + 1
    
    # Plot dell'immagine originale
    plt.figure(figsize=(12, 6))
    
    # Plot dell'immagine originale
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Image: {image_name}\nFolder: {folder}")
    
    
    # Plot dei valori singolari
    plt.subplot(1, 3, 2)
    plt.plot(S, label="Singular value")
    plt.axvline(k, color='red', linestyle='--', label=f"k = {k}")
    plt.title(f"Singular value of {image_name}")
    plt.xlabel("Index singular value")
    plt.ylabel("Singular value")
    plt.legend()
    
    # Plot dell'energia cumulativa
    plt.subplot(1, 3, 3)
    plt.plot(cumulative_energy, label="Cumulative energy")
    plt.axvline(k, color='red', linestyle='--', label=f"k = {k}")
    plt.title(f"Cumulative energy\nThreshold: {energy_threshold * 100:.1f}%")
    plt.xlabel("Index singular value")
    plt.ylabel("Cumulative energy")
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [28]:
import os
import cv2
import numpy as np

# Funzione per applicare la SVD e visualizzare i risultati
def images_svd(dataset_path, energy_threshold=0.99):
   
    images = []  # Lista per salvare le immagini
    labels = []  # Lista per salvare le etichette
    
    # Itera attraverso i file del dataset
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        
        if os.path.isdir(folder_path):  # Se è una directory (persona)
            for image_name in os.listdir(folder_path):
                
                image_path = os.path.join(folder_path, image_name)
                
                # Carica l'immagine in scala di grigi
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                
                if img is not None:
                    # Normalizzare l'immagine
                    img = img / 255.0  # Normalizza i valori dei pixel in [0, 1]
                    
                    # Reshaping per la SVD (ogni immagine diventa un vettore 1D)
                    
                    # Calcola la SVD
                    U, S, Vt = np.linalg.svd(img, full_matrices=False)
                    
                    # Calcola l'energia totale
                    
                    # plotting(S, energy_threshold, img, image_name, folder)
                    
                    # Salva l'immagine e l'etichetta
                    images.append(img.flatten())  # Aggiungi l'immagine alla lista
                    labels.append(folder)  # Aggiungi l'etichetta alla lista
                    
    return np.array(images), np.array(labels)


In [29]:
# Percorso del dataset (modifica con il tuo percorso)
dataset_path = 'Kaggle/'  # Cambia con il percorso effettivo del tuo dataset

# Esegui la funzione
images, labels= images_svd(dataset_path)
images.shape
labels.shape

(400,)

In [32]:
# Divisione train/test
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42,stratify=labels)

# Applicazione SVD
U, S, Vt = np.linalg.svd(X_train, full_matrices=False)
k = 10  # Numero di componenti principali
X_train_reduced = np.dot(X_train, Vt.T[:, :k])  # Proiezione nello spazio ridotto
X_test_reduced = np.dot(X_test, Vt.T[:, :k])

# Classificatore (K-Nearest Neighbors in questo caso)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_reduced, y_train)

# Predizione
y_pred = knn.predict(X_test_reduced)

# Valutazione
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 91.25%

Classification Report:
               precision    recall  f1-score   support

          s1       1.00      1.00      1.00         2
         s10       1.00      1.00      1.00         2
         s11       1.00      1.00      1.00         2
         s12       0.67      1.00      0.80         2
         s13       1.00      1.00      1.00         2
         s14       1.00      1.00      1.00         2
         s15       0.67      1.00      0.80         2
         s16       1.00      0.50      0.67         2
         s17       0.67      1.00      0.80         2
         s18       1.00      1.00      1.00         2
         s19       1.00      1.00      1.00         2
          s2       1.00      1.00      1.00         2
         s20       1.00      1.00      1.00         2
         s21       0.50      1.00      0.67         2
         s22       1.00      1.00      1.00         2
         s23       0.50      0.50      0.50         2
         s24       1.00      1.00      